## Algoritmo de machine learning para detección de párkinson - XGBOOST

In [ ]:
#Instalación de la librería XGBoost
pip install xgboost

In [ ]:
#1. Carga de paquetes 
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import os
import matplotlib.pyplot as plt
import matplotlib.dates as md
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
#Esto permite generar gráficas interactivas
%matplotlib qt

In [ ]:
# Esta función calcula el gasto de memoria en MB
def memory_usage_psutil():
    import psutil
    import os
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / float(2 ** 20)
    return mem

In [ ]:
#Ajuste del directorio - (Al trabajar con un disco externo se requiere ajustar el directorio dependiendo del ordenador en el que se trabaje)
#Si se trabaja desde el odenador propio correr esta celda
path = 'C:\\Users\\Vaneza'
os.chdir(path)
os.getcwd()
disco = 'E'

In [ ]:
#Si se trabaja desde el ordenador del trabajo correr esta celda
path = 'C:\\Users\\Y8764982H\\OneDrive - Generalitat de Catalunya/TFG/'
os.chdir(path)
os.getcwd()
disco = 'D'

In [ ]:
#aplplicación de la función de memoria
memory_usage_psutil()

## Carga de archivos

In [ ]:
#Datos reales
all_CT_PD_30s_Real= pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\segmented_tables\\all_CT_PD_30s_39ch.csv'%(disco))
#Datos sintéticos
all_CT_PD_30s_Syn= pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_A.csv'%(disco))

### 1. Exploración de los datos

In [ ]:
#Generar un papa de calor de las correlaciones entre las columnas de los datos 
corr = model_df.drop(columns=['Patient','Time','Diagnosis']).corr()
ax = sns.heatmap(corr.iloc[:39, :39], linewidth=0.5)
plt.show()

In [ ]:
#Corregir los nombres de las variables
model_df = all_CT_PD_30s_Syn.copy().drop(columns=['Unnamed: 0'])
model_df_real = all_CT_PD_30s_Real.copy()

#Cambiar la columna de tiempo de Datetime a float (volver al vector de 0 a 30s)
rng = np.arange(0.000, 30, 0.002)
rng = pd.DataFrame(rng,columns=['Time'])
#print(rng)
#print(len(rng))
Time_df = rng
for i in range(0,111):
    Time_df = pd.concat([Time_df,rng],axis=0).reset_index(drop=True)
print(Time_df)  #Nuevo vector del tiempo
#-------------------------------------------------------------------------------
#Reemplazar la columna de tiempo por el nuevo vector

model_df_real['Time'] = Time_df
model_df_real

In [ ]:
#Información dek dataframe
print(model_df_real.info())

## Modelo XGBoost

El siguiente conjunto de código solo se ejecuta cuando se estén evaluando dato sintéticos

### Unión de los datos sintéticos con una porción de los datos reales

In [ ]:
#Primero se seleccionan aleatoriamente los pacientes para luego extraer todos los registros de este
random_real = model_df_real.copy()
random_real = random_real.sample(n = 86, replace=False, random_state = 2).drop_duplicates()
random_real = list(dict.fromkeys(random_real['Patient']))
len(random_real)

In [ ]:
#Extraemos las filas de los pacientes escogidos aleatoriamente
random_real_df = []
for n in range(len(random_real)):
    if n == 0:
        random_real_df = model_df_real.loc[model_df_real['Patient'] == random_real[n]]
    else:
        random_real_df1 = model_df_real.loc[model_df_real['Patient'] == random_real[n]]
        random_real_df = pd.concat([random_real_df,random_real_df1],axis=0).reset_index(drop=True)  
random_real_df

In [ ]:
##Eliminar los pacientes extraídos del dataset real para no genera overfitting
real = model_df_real.copy(deep=True)
for n in random_real:
    indexes = real.loc[real['Patient'] == n]
    real = real.drop(index=indexes.index, axis=0)
real = real.reset_index(drop=True)
real

In [ ]:
## Concatenar aleatoriamente los datos reales con los sintéticos
syn_real_model_df = pd.concat([model_df,random_real_df],axis=0).reset_index(drop=True)
syn_real_model_df

In [ ]:
#Creación de nombres de pacientes para crear ventanas de 30 segundos
for i in range(0,300):
    patient = ['Patient_%s'%(i)] * 15000
    if (i == 0):
        patient1 = pd.DataFrame(patient,columns=['Patient'])
    else:
        patient1_t = pd.DataFrame(patient,columns=['Patient'])
        patient1 = pd.concat([patient1,patient1_t],axis=0).reset_index(drop=True)
        
patient1

In [43]:
syn_real_model_df['Patient'] = patient1
syn_real_model_df

,Patient,Diagnosis,Time,Fz,FC1,C3,T7,CP5,CP1,P3,...,POz,PO4,PO8,P6,P2,CPz,CP4,TP8,C6,C2
0,Patient_0,0,0.000,-0.003063,0.532317,0.001482,-3.656705,0.003446,1.835564,-1.143459,...,4.138678,-6.841405,0.004125,2.559477,-0.520981,-3.778777,-4.604167,0.002650,-11.473443,5.263293
1,Patient_0,0,0.002,0.384354,-9.423356,-2.123312,3.451568,0.037242,-0.264607,3.733971,...,1.034917,1.692262,-2.393040,5.534984,0.422182,-2.865445,-3.591982,0.002650,3.438419,-7.450541
2,Patient_0,0,0.004,-10.426778,-0.000855,-4.058763,-1.748821,-1.910042,1.578725,-2.354752,...,-0.694865,6.356311,-2.532005,1.142355,0.443890,-5.152515,4.297418,-2.462974,-9.122838,-7.383072
3,Patient_0,0,0.006,-3.071366,-5.627454,-2.771687,0.005552,0.137462,-2.914209,2.338753,...,2.106869,4.835685,-1.696615,0.545445,0.835161,-4.528877,-6.043903,0.002650,-8.101640,-0.004458
4,Patient_0,0,0.008,-7.063727,-6.534853,-5.473887,1.252795,0.461665,-2.900879,5.545088,...,2.345254,2.294672,-1.734597,1.076149,-1.882763,-2.659847,-2.922985,-8.698761,-9.101163,-3.242604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499995,Patient_299,1,29.990,21.100325,13.556267,4.901668,1.927698,-6.502246,-0.021275,-0.693315,...,-0.391076,6.867064,5.948630,1.295977,0.076105,0.182356,3.987044,4.721860,17.293222,2.710637
4499996,Patient_299,1,29.992,21.195600,13.716537,5.195432,0.334844,-6.742901,0.382840,-0.434173,...,-0.213212,6.677945,5.843429,1.497851,0.291411,0.365396,4.287427,4.614795,17.801438,3.305194
4499997,Patient_299,1,29.994,20.921385,13.773670,5.602830,-0.685703,-6.430155,0.670713,-0.212303,...,-0.002269,6.177276,5.681112,1.582168,0.471357,0.515938,4.492207,4.274391,17.920972,3.650496
4499998,Patient_299,1,29.996,20.290689,13.616093,6.075878,-0.926390,-5.591634,0.750814,-0.108663,...,0.209969,5.420653,5.440293,1.510845,0.569561,0.604439,4.494166,3.755393,17.528928,3.682801


### 2. Preparación de los datos Reales

In [ ]:
# Features - Características para entrenar el modelo
features = model_df_real.iloc[:,1:]
# Label - Columna de diagnóstico
diagnosis = model_df_real.iloc[:,0]

In [ ]:
diagnosis

### 2. Preparación de los datos sintéticos

In [ ]:
# Features - Características para entrenar el modelo
features = syn_real_model_df.iloc[:,2:]
# Label - Columna de diagnóstico de los datos sintéticos
diagnosis = syn_real_model_df.iloc[:,1]
diagnosis

In [20]:
# Features - exclude Patient and diagnosis columns
features_Real = real.iloc[:,2:]
# Label -Columna de diagnóstico de los datos reales
diagnosis_Real = real.iloc[:,1]
diagnosis_Real

0         0
1         0
2         0
3         0
4         0
         ..
779995    1
779996    1
779997    1
779998    1
779999    1
Name: Diagnosis, Length: 780000, dtype: int64

### 3. División de los datos

La siguiente celda solo se ejecuta al evaluar solo datos reales

In [ ]:
#Separación de los datos en entrenamiento (Train) y prueba (Test)
features_train, features_test, diagnosis_train, diagnosis_test = train_test_split(features, diagnosis, test_size=0.2, random_state=557) 

### 4. Definición del modelo

In [ ]:
# Definición del modelo con los primeros parámetros - Datos reales
creditxgb = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=10,
 min_child_weight=50,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic'
)

### 5. Entrenamiento del modelo

In [ ]:
# Implementación del modelo con los parámetros preliminares
creditxgb.fit(features_train,diagnosis_train)

### 6. Evaluación del modelo

In [ ]:
# Evaluar las probabilidades del diagnóstico
pred_prob = creditxgb.predict_proba(features_train)
pred_prob_1 = pred_prob[:,1]
np.quantile(pred_prob_1,0.88)
pred_class = np.where(pred_prob_1>=0.33,1,0)

In [ ]:
#Esta celda calcula la exactitud de la predicción sobre los datos de entrenamiento
accuracy = accuracy_score(diagnosis_train, pred_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(mean_absolute_error(diagnosis_train, pred_class))

### 7. Afinamiento del modelo

In [ ]:
# Parámetros nuevos para agenerar un modelo óptimo
Opticreditxgb = xgb.XGBClassifier(
 learning_rate =0.3,
 n_estimators=100,
 max_depth=15,
 min_child_weight=6,
 gamma=0.0,
 subsample=0.9,
 colsample_bytree=0.6,
 objective= 'binary:logistic'
)

#Volvemos a entrenar el modelo con los nuevos parámetros
Opticreditxgb.fit(features_train,diagnosis_train)

### 8.1 Evaluación final sobre los datos de entrenamiento

In [ ]:
#Si se entrena el modelo con los datos reales:
features_train = features_train
diagnosis_train = diagnosis_train
features_test = features_test
diagnosis_test = diagnosis_test

In [ ]:
#Si se entrena el modelo con los datos sintéticos:
features_train = features
diagnosis_train = diagnosis
features_test = features_Real
diagnosis_test = diagnosis_Real

In [ ]:
#Esta celda imprime la gráfica de importancia de características
xgb.plot_importance(Opticreditxgb)
plt.rcParams['figure.figsize'] = [5, 5]
plt.title('Importancia de las características - Señales Reales *')
plt.xlabel('Puntuación')
plt.ylabel('Características')
plt.show()

In [ ]:
# Evaluar las probabilidades del diagnóstico
pred_prob = Opticreditxgb.predict_proba(features_train)
pred_prob_1 = pred_prob[:,1]
np.quantile(pred_prob_1,0.88)
#Para datos reales:
pred_class = np.where(pred_prob_1>=0.33,1,0)
#Para datos sintéticos:
pred_class = np.where(pred_prob_1>=0.5,1,0)

In [ ]:
#Esta celda calcula la exactitud de la predicción sobre los datos de entrenamiento e 
#imprime los verdaderos y falsos positivos y negativos
accuracy = accuracy_score(diagnosis_train, pred_class)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

cnf_matrix = metrics.confusion_matrix(diagnosis_train, pred_class)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Matriz de confusión - Train')
plt.ylabel('Diagnóstico Real')
plt.xlabel('Diagnóstico Predicho')

In [ ]:
#Con esta celda se calcula el error cuadrado medio 
print(mean_absolute_error(diagnosis_train, pred_class))

In [ ]:
#Reporte de clasificación
print(classification_report(diagnosis_train, pred_class))

In [ ]:
# Curva ROC que evalua el rendimiento del modelo
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Tasa de falsos positivos')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.title('Curva ROC - Datos reales - Train')
    plt.legend()
    plt.show()
    
fpr_dev, tpr_dev, thresholds_dev =roc_curve(diagnosis_train,pred_class)
plot_roc_curve(fpr_dev, tpr_dev)
plt.show()

### 8.2 Evaluación sobre los datos de prueba


In [ ]:
# Evaluar las probabilidades del diagnóstico
pred_prob_testing = Opticreditxgb.predict_proba(features_test)
pred_prob_test_1 = pred_prob_testing[:,1]
np.quantile(pred_prob_test_1,0.88)
#Para datos reales:
pred_class_testing = np.where(pred_prob_test_1>=0.33,1,0)
#Para datos sintéticos:
pred_class_testing = np.where(pred_prob_test_1>=0.6,1,0)

In [ ]:
#Esta celda calcula la exactitud de la predicción sobre los datos de prueba e 
#imprime los verdaderos y falsos positivos y negativos
accuracy = accuracy_score(diagnosis_test, pred_class_testing)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

cnf_matrix = metrics.confusion_matrix(diagnosis_test, pred_class_testing)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Matriz de confusión - Test')
plt.ylabel('Diagnóstico Real')
plt.xlabel('Diagnóstico Predicho')

In [ ]:
print(mean_absolute_error(diagnosis_test, pred_class_testing))

In [ ]:
#Reporte de clasificación
print(classification_report(diagnosis_test, pred_class_testing))

In [ ]:
# Curva ROC que evalua el rendimiento del modelo
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Tasa de falsos positivos')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.title('Curva ROC - Datos reales - Test')
    plt.legend()
    plt.show()

fpr_dev, tpr_dev, thresholds_dev =roc_curve(diagnosis_test,pred_class_testing)
plot_roc_curve(fpr_dev, tpr_dev)
plt.show()